In [ ]:
'''

# Carregando dados
# Modelo de Probabilidade de Sinistro
X_train, X_test, y_train, y_test (y = categoria)
## Linear Pipeline
cat_pipeline
    - OHE
    - UMAP
num_pipeline
    - SCA
full_pipeline
    - cat_pipeline
    - num_pipeline
    - LinearModel
# Modelo de Valor do Sinistro


## FILTRAR APENAS SINISTROS
X_train_sin, X_test_sin, y_train_sin, y_test_sin (y = valor)
## Dataprep Pipeline
cat_pipeline_nn
    - OHE
num_pipeline_nn
    - SCA
X_train_trans_num
X_train_trans_cat
X_test_trans_num
X_test_trans_cat
## Rede Neural

# Avaliando Modelo
pred_prob = lm.predict_proba(X_test)
pred_valor = nn.predict([cat_pipeline.transform(X_test[cat_vars])])
valor_proj = pred_prob * pred_valor
'''

In [95]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
import umap
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error


In [2]:
#reading the info

insurance = pd.read_csv("data\\insurance_train_set.csv")

# Understanding the data

In [3]:
import sklearn

In [4]:
insurance.head()

,Row_ID,Household_ID,Vehicle,Calendar_Year,Model_Year,Blind_Make,Blind_Model,Blind_Submodel,Cat1,Cat2,...,Var5,Var6,Var7,Var8,NVCat,NVVar1,NVVar2,NVVar3,NVVar4,Claim_Amount
0,1,1,3,2005,2005,K,K.78,K.78.2,D,C,...,1.008912,0.261040,0.907793,-0.077998,M,-0.23153,-0.266117,-0.272337,-0.251419,0.0
1,2,2,2,2005,2003,Q,Q.22,Q.22.3,B,C,...,1.240851,0.432987,-0.726459,0.204785,O,-0.23153,-0.266117,-0.272337,-0.251419,0.0
2,3,3,1,2005,1998,AR,AR.41,AR.41.1,B,?,...,-0.971487,-1.405797,-0.837048,-1.176858,F,-0.23153,-0.266117,-0.272337,-0.251419,0.0
3,4,3,1,2006,1998,AR,AR.41,AR.41.1,B,?,...,-0.971487,-1.405797,-0.837048,-1.176858,F,-0.23153,-0.266117,-0.272337,-0.251419,0.0
4,5,3,2,2005,2001,D,D.20,D.20.0,J,C,...,0.812656,2.112691,1.534462,2.347260,F,-0.23153,-0.266117,-0.272337,-0.251419,0.0


In [5]:
insurance.shape

(13184290, 35)

In [6]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13184290 entries, 0 to 13184289
Data columns (total 35 columns):
 #   Column          Dtype  
---  ------          -----  
 0   Row_ID          int64  
 1   Household_ID    int64  
 2   Vehicle         int64  
 3   Calendar_Year   int64  
 4   Model_Year      int64  
 5   Blind_Make      object 
 6   Blind_Model     object 
 7   Blind_Submodel  object 
 8   Cat1            object 
 9   Cat2            object 
 10  Cat3            object 
 11  Cat4            object 
 12  Cat5            object 
 13  Cat6            object 
 14  Cat7            object 
 15  Cat8            object 
 16  Cat9            object 
 17  Cat10           object 
 18  Cat11           object 
 19  Cat12           object 
 20  OrdCat          object 
 21  Var1            float64
 22  Var2            float64
 23  Var3            float64
 24  Var4            float64
 25  Var5            float64
 26  Var6            float64
 27  Var7            float64
 28  Var8      

In [ ]:
insurance.isna().sum()

In [ ]:
for i in insurance.columns:
    print(insurance[i].value_counts())

In [ ]:
for i in insurance.columns:
    print(insurance[i].unique())

In [ ]:
insurance["Blind_Make"].value_counts()

In [ ]:
insurance["Blind_Model"].value_counts()

In [ ]:
insurance["Blind_Submodel"].value_counts()

In [ ]:
insurance["Claim_Amount"].value_counts()

# Preparing the data

In [7]:
#creating yeas of insurance:

insurance["years_insurance"] = 2007 - insurance["Calendar_Year"]



In [8]:
#creating cars's age

insurance["age_car"] = 2007 - insurance["Model_Year"]


In [9]:
insurance["Cat12"] = insurance["Cat12"].replace(np.nan,"?")

In [10]:
insurance['OrdCat_cat'] = insurance['OrdCat'].astype("string")

In [11]:
insurance["claimed"] = insurance["Claim_Amount"].apply(lambda x : 1 if x>0 else 0)

In [12]:
insurance_claimed = insurance[insurance["claimed"]==1]

In [13]:
insurance = insurance.sample(100000)

In [14]:
cat_vars = ['Blind_Make', 'Blind_Model', 'Blind_Submodel', 'Cat1', 'Cat2', 'Cat3',
       'Cat4', 'Cat5', 'Cat6', 'Cat7', 'Cat8', 'Cat9', 'Cat10', 'Cat11',
       'Cat12','OrdCat_cat','NVCat']

num_vars = ['Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6',
       'Var7', 'Var8','NVVar1', 'NVVar2', 'NVVar3', 'NVVar4',
          'years_insurance', 'age_car']

y_var_claimed = 'claimed'
y_var_value = "Claim_Amount"

# Creating probability of claim model


In [15]:
X_prob = insurance[num_vars + cat_vars]
y_prob = insurance["claimed"]

X_train_prob, X_test_prob, y_train_prob, y_test_prob = train_test_split(X_prob, y_prob, test_size=0.2)

In [16]:
# Pipeline Numérico
sca = StandardScaler()
lista_etapas = [("STD_SCALER", sca)]
num_pipeline = Pipeline(lista_etapas)

# Pipeline Categórico
ohe = OneHotEncoder(drop = 'first', handle_unknown="ignore")
umap_fit_5 = umap.UMAP(n_neighbors = 5, n_components=25, random_state=42)
cat_pipeline = Pipeline([("ONE_HOT", ohe), ("UMAP",umap_fit_5)])

# Pipeline de Preprocessamento
pipeline_preprocessamento = [
    ("NUMERICOS", num_pipeline, num_vars),
    ("CATEGORICOS", cat_pipeline, cat_vars),
]
data_prep_pipeline = ColumnTransformer(pipeline_preprocessamento)

In [31]:
lm = LogisticRegression(class_weight = 'balanced')
lm_pipeline = Pipeline([("PREP", data_prep_pipeline), ("LM", lm)])
lm_pipeline.fit(X_train_prob, y_train_prob)

Pipeline(steps=[('PREP',
                 ColumnTransformer(transformers=[('NUMERICOS',
                                                  Pipeline(steps=[('STD_SCALER',
                                                                   StandardScaler())]),
                                                  ['Var1', 'Var2', 'Var3',
                                                   'Var4', 'Var5', 'Var6',
                                                   'Var7', 'Var8', 'NVVar1',
                                                   'NVVar2', 'NVVar3', 'NVVar4',
                                                   'years_insurance',
                                                   'age_car']),
                                                 ('CATEGORICOS',
                                                  Pipeline(steps=[('ONE_HOT',
                                                                   OneHotEncoder(drop='first',
                                                               

# Creating  model of value of claim


In [23]:
X_value_claimed = insurance_claimed[num_vars + cat_vars]
y_value_claimed = insurance_claimed["Claim_Amount"]

X_train_value, X_test_value, y_train_value, y_test_value = train_test_split(X_value_claimed, y_value_claimed, test_size=0.2)

In [24]:
# Pipeline Numérico
sca = StandardScaler()
lista_etapas = [("STD_SCALER", sca)]
num_pipeline = Pipeline(lista_etapas)

# Pipeline Categórico
ohe = OneHotEncoder(drop = 'first', sparse = False,handle_unknown="ignore")
lista_etapa_cat2 = [("ONE_HOT", ohe)]
cat_pipeline = Pipeline(lista_etapa_cat2)

In [117]:
# Pipeline Numérico
sca = StandardScaler()
lista_etapas = [("STD_SCALER", sca)]
num_pipeline = Pipeline(lista_etapas)

# Pipeline Categórico
ohe = OneHotEncoder(drop = 'first', handle_unknown="ignore")
umap_fit_5 = umap.UMAP(n_neighbors = 5, n_components=25, random_state=42)
cat_pipeline = Pipeline([("ONE_HOT", ohe), ("UMAP",umap_fit_5)])

# Pipeline de Preprocessamento
pipeline_preprocessamento = [
    ("NUMERICOS", num_pipeline, num_vars),
    ("CATEGORICOS", cat_pipeline, cat_vars),
]
data_prep_pipeline2 = ColumnTransformer(pipeline_preprocessamento)

In [118]:
lm = LinearRegression()
lm_pipeline_value = Pipeline([("PREP", data_prep_pipeline2), ("LM", lm)])
lm_pipeline_value.fit(X_train_value, y_train_value)

Pipeline(steps=[('PREP',
                 ColumnTransformer(transformers=[('NUMERICOS',
                                                  Pipeline(steps=[('STD_SCALER',
                                                                   StandardScaler())]),
                                                  ['Var1', 'Var2', 'Var3',
                                                   'Var4', 'Var5', 'Var6',
                                                   'Var7', 'Var8', 'NVVar1',
                                                   'NVVar2', 'NVVar3', 'NVVar4',
                                                   'years_insurance',
                                                   'age_car']),
                                                 ('CATEGORICOS',
                                                  Pipeline(steps=[('ONE_HOT',
                                                                   OneHotEncoder(drop='first',
                                                               

# Creating the final model


In [112]:
y_pred_lm_proba = lm_pipeline.predict_proba(X_test_prob)

In [113]:
y_pred_lm_value = lm_pipeline_value.predict(X_test_prob)

In [114]:
X_test_prob["claim_prob"] = y_pred_lm_proba[:, 1]
X_test_prob["claim_prob_50"] = X_test_prob["claim_prob"].apply(lambda x: 1 if x>0.50 else 0)
X_test_prob["claim_prob_60"] = X_test_prob["claim_prob"].apply(lambda x: 1 if x>0.60 else 0)
X_test_prob["claim_prob_70"] = X_test_prob["claim_prob"].apply(lambda x: 1 if x>0.70 else 0)
X_test_prob["claim_prob_735"] = X_test_prob["claim_prob"].apply(lambda x: 1 if x>0.735 else 0)
X_test_prob["claim_prob_80"] = X_test_prob["claim_prob"].apply(lambda x: 1 if x>0.80 else 0)
X_test_prob["claim_prob_90"] = X_test_prob["claim_prob"].apply(lambda x: 1 if x>0.90 else 0)

X_test_prob["claim_value"] = y_pred_lm_value


X_test_prob["claimed_paied_50"] = X_test_prob["claim_prob_50"]*X_test_prob["claim_value"]
X_test_prob["claimed_paied_60"] = X_test_prob["claim_prob_60"]*X_test_prob["claim_value"]
X_test_prob["claimed_paied_70"] = X_test_prob["claim_prob_70"]*X_test_prob["claim_value"]
X_test_prob["claimed_paied_735"] = X_test_prob["claim_prob_735"]*X_test_prob["claim_value"]
X_test_prob["claimed_paied_80"] = X_test_prob["claim_prob_80"]*X_test_prob["claim_value"]
X_test_prob["claimed_paied_90"] = X_test_prob["claim_prob_90"]*X_test_prob["claim_value"]

X_test_prob["y"] = insurance.loc[X_test_prob.index,"Claim_Amount"]

In [115]:
print(X_test_prob["y"].sum())
print("")
print(X_test_prob["claimed_paied_50"].sum())
print(X_test_prob["claimed_paied_60"].sum())
print(X_test_prob["claimed_paied_735"].sum())
print(X_test_prob["claimed_paied_70"].sum())
print(X_test_prob["claimed_paied_80"].sum())
print(X_test_prob["claimed_paied_90"].sum())

24940.502093590003

1429224.193119984
294231.38320420304
30704.667366974652
55542.25566248548
11225.443553527253
221.88613946201366


In [119]:
print(round(np.sqrt(mean_squared_error(X_test_prob["claimed_paied_50"],X_test_prob["y"] )), 2))
print(round(np.sqrt(mean_squared_error(X_test_prob["claimed_paied_60"],X_test_prob["y"] )), 2))
print(round(np.sqrt(mean_squared_error(X_test_prob["claimed_paied_70"],X_test_prob["y"] )), 2))
print(round(np.sqrt(mean_squared_error(X_test_prob["claimed_paied_735"],X_test_prob["y"] )), 2))
print(round(np.sqrt(mean_squared_error(X_test_prob["claimed_paied_80"],X_test_prob["y"] )), 2))
print(round(np.sqrt(mean_squared_error(X_test_prob["claimed_paied_90"],X_test_prob["y"] )), 2))

125.01
56.5
33.5
30.44
27.93
26.4


In [120]:
print(round(np.sqrt(mean_squared_error(X_test_prob["claimed_paied_50"],X_test_prob["y"] )), 2))
print(round(np.sqrt(mean_squared_error(X_test_prob["claimed_paied_60"],X_test_prob["y"] )), 2))
print(round(np.sqrt(mean_squared_error(X_test_prob["claimed_paied_70"],X_test_prob["y"] )), 2))
print(round(np.sqrt(mean_squared_error(X_test_prob["claimed_paied_735"],X_test_prob["y"] )), 2))
print(round(np.sqrt(mean_squared_error(X_test_prob["claimed_paied_80"],X_test_prob["y"] )), 2))
print(round(np.sqrt(mean_squared_error(X_test_prob["claimed_paied_90"],X_test_prob["y"] )), 2))


125.01
56.5
33.5
30.44
27.93
26.4
